<a href="https://colab.research.google.com/github/Ecological-Complexity-Lab/link_prediction_project/blob/main/MLmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [76]:
# TODO : use this dataset
df_mandena = pd.read_csv('/content/drive/MyDrive/LinkPredictionProject/Data/ML_rattus_mandena.csv', encoding='latin1')
df_mandena.head()

,host_ID,asv_ID,Link,grid,pca_grid_attr,pca_grid_sm,season,elevation.obs,mass,sex,...,host_degree,host_betweenness,host_closeness,host_module,asv_degree,asv_betweenness,asv_closeness,asv_module,shortest_path,pref_attach
0,2010,ASV_1,1,brushy_regrowth,-0.197466,-2.224026,1,144.218796,80.0,female,...,12,0.003539,0.003651,1,129,0.050775,0.003051,1,1,1548
1,2011,ASV_1,1,brushy_regrowth,-0.197466,-2.224026,1,145.284958,51.0,male,...,13,0.002761,0.003479,1,129,0.050775,0.003051,1,1,1677
2,2013,ASV_1,0,brushy_regrowth,-0.197466,-2.224026,1,118.188316,88.0,female,...,18,0.004112,0.003691,1,129,0.050775,0.003051,1,3,2322
3,2014,ASV_1,1,brushy_regrowth,-0.197466,-2.224026,1,133.551437,77.0,male,...,20,0.004784,0.003758,1,129,0.050775,0.003051,1,1,2580
4,2015,ASV_1,1,brushy_regrowth,-0.197466,-2.224026,1,145.232788,65.0,female,...,6,0.004310,0.003472,1,129,0.050775,0.003051,1,1,774


In [71]:
df = pd.read_csv('/content/drive/MyDrive/LinkPredictionProject/Data/data_asv_rra0.001_p0.01_th5000.csv', encoding='latin1')
df.head()

,village,asv_ID,host_p,host_ID,host_species,grid,season,reads,total_reads
0,Andatsakala,ASV_1,0.794872,3500,Rattus rattus,brushy_regrowth,1,0.036904,28046.0
1,Andatsakala,ASV_1,0.794872,3501,Rattus rattus,brushy_regrowth,1,0.006760,30621.0
2,Andatsakala,ASV_1,0.794872,3502,Rattus rattus,brushy_regrowth,1,0.424856,53517.0
3,Andatsakala,ASV_1,0.794872,3503,Rattus rattus,brushy_regrowth,1,0.197985,39695.0
4,Andatsakala,ASV_1,0.794872,3516,Rattus rattus,brushy_regrowth,1,0.099647,31722.0


## Data preprocessing

In [72]:
# convert df['reads'] to binary
reads_threshold = df['reads'].mean()
df['reads'] = df['reads'].apply(lambda x: 1 if x > reads_threshold else 0)
# work only on Mandena village
hosts_df = df[df["village"] == "Mandena"]
hosts_df.drop(["village", "host_species", "asv_ID", "host_ID", "host_p", "total_reads"], axis=1, inplace=True)
hosts_df.head()

<ipython-input-72-848a5faa40c0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosts_df.drop(["village", "host_species", "asv_ID", "host_ID", "host_p", "total_reads"], axis=1, inplace=True)


,grid,season,reads
28813,brushy_regrowth,1,1
28814,brushy_regrowth,1,1
28815,brushy_regrowth,1,1
28816,brushy_regrowth,1,1
28817,brushy_regrowth,1,0


In [58]:
# get dummies for host_species, grid, season
dummies = pd.get_dummies(data=hosts_df, columns=["grid", "season"])
dummies.head()
# dummies = dummies.drop(["host_species", "grid", "season"], axis=1)
# dummies = pd.concat([df, dummies], axis=1)
# dummies.head()

,asv_ID,host_p,host_ID,reads,host_species_Rattus rattus,grid_agriculture,grid_agroforest,grid_brushy_regrowth,grid_flooded_rice,grid_secondary_forest,grid_semi-intact_forest,grid_village,season_1,season_2,season_3
28813,ASV_1,0.674419,2010,1,True,False,False,True,False,False,False,False,True,False,False
28814,ASV_1,0.674419,2011,1,True,False,False,True,False,False,False,False,True,False,False
28815,ASV_1,0.674419,2014,1,True,False,False,True,False,False,False,False,True,False,False
28816,ASV_1,0.674419,2015,1,True,False,False,True,False,False,False,False,True,False,False
28817,ASV_1,0.674419,2016,0,True,False,False,True,False,False,False,False,True,False,False


## Choosing a model

In [35]:

# X = df.drop(["reads", "asv_ID", "host_ID", "host_species", "host_p"], axis=1).values
# y = df["reads"].values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# models = {
#   "Logistic Regression": LogisticRegression(),
#   "KNN": KNeighborsClassifier(),
#   "Random Forest": RandomForestClassifier()
# }
# results = []

# for model in models.values():
#   kf = KFold(n_splits=5, shuffle=True, random_state=42)
#   cv_results = cross_val_score(model, X_train, y_train, cv=kf)
#   results.append(cv_results)

# plt.boxplot(results, labels=models.keys())
# plt.show()

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1196, in fit
    X, y = self._validate_data(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py", line 1106, in check_X_y
    X = check_array(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py", line 879, 